In [27]:
# Force restart and reimport
import importlib
import sys
if 'nirs4all' in sys.modules:
    del sys.modules['nirs4all']
if 'nirs4all.pipeline.runner' in sys.modules:
    del sys.modules['nirs4all.pipeline.runner']

In [28]:
%load_ext autoreload
%autoreload 2
%reset -f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_decomposition import PLSRegression

from nirs4all.operators.transformations import Gaussian, SavitzkyGolay, StandardNormalVariate, Haar
from nirs4all.pipeline.config import PipelineConfigs
from nirs4all.dataset.dataset_config import DatasetConfigs
from nirs4all.pipeline.runner import PipelineRunner
import json

pipeline = [
    # Normalize the spectra reflectance
    MinMaxScaler(feature_range=(0.1, 0.8)),

    # Generate 10 version of feature augmentation combinations (3 elements with size 1 to 2, ie. [SG, [SNV, GS], Haar])
    {
        "feature_augmentation": {
            "_or_": [
                Gaussian, StandardNormalVariate, SavitzkyGolay, Haar,
            ],
            "size": [3, (1,2)],
            "count": 2,
        }
    },

    # Split the dataset in train and validation
    ShuffleSplit(n_splits=3, test_size=.25),

    # Normalize the y values
    {"y_processing": MinMaxScaler()},

    # PLS regression with 1 to 60 components
    {
        "model": PLSRegression(),
        # "model_params": {
        #     "n_components": {
        #         "_range_": [1, 60],
        #     }
        # }
    }
]

# create pipeline config
config = PipelineConfigs(pipeline)

# print(config.has_configurations)
# print(config)
# print(PipelineConfigs.value_of_str(config.steps, "n_components"))

path = ['../../sample_data/regression', '../../sample_data/classification', '../../sample_data/binary']
# # path = '../../sample_data/regression'
dataset_config_obj = DatasetConfigs(path)

runner = PipelineRunner()
results = runner.run(config, dataset_config_obj)

# print(f"Pipeline execution completed! Got {len(results)} results")
# for i, (ds, hist, _) in enumerate(results):
    # print(f"Result {i+1}: {ds.name}")

Configuration generates 2 configurations.
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
Loading dataset:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
📊 Dataset: regression
Features (samples=189, sources=1):
- Source 0: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
Targets: (samples=189, targets=1, processings=['numeric'])
- numeric: min=1.33, max=128.31, mean=30.779
Indexes:
- "train", ['raw']: 130 samples
- "test", ['raw']: 59 samples
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
Loading dataset:
---------------------------------------------------------------------------------------------------------------------

⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
Loading dataset:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
📊 Dataset: regression
Features (samples=189, sources=1):
- Source 0: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
Targets: (samples=189, targets=1, processings=['numeric'])
- numeric: min=1.33, max=128.31, mean=30.779
Indexes:
- "train", ['raw']: 130 samples
- "test", ['raw']: 59 samples
regression
Pipeline execution completed! Got 0 results
